In [1]:
import torch as t
from torch import nn
import numpy as np
import pandas as pd

from utils import Pipeline_lstm
from torch.utils.tensorboard import SummaryWriter

import warnings
warnings.simplefilter('ignore')
DEVICE = 'cuda' if t.cuda.is_available() else 'cpu'

In [2]:
df = pd.read_csv('./train.csv', index_col=0)
df_test = pd.read_csv('./test.csv', index_col=0)

df_coin = df.loc[df['coin_id'] == 'coin_1']

In [3]:
features = ['feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_8', 'feat_9', 'feat_10']
lookback=30

train_split = 0.8
split_ind = int(df_coin.shape[0] * train_split)

X_train = df_coin[features].iloc[:split_ind]
X_test = df_coin[features].iloc[split_ind:]

y_train = df_coin['fwd_ret_3d'].iloc[:split_ind]
y_test = df_coin['fwd_ret_3d'].iloc[split_ind:]

pipe = Pipeline_lstm(lookback=lookback)
train_ds = pipe.fit_transform(X_train, y_train)
val_ds = pipe.transform(X_test, y_test)

In [4]:
class ConvModel(nn.Module):
    def __init__(self, channels=5):
        super().__init__()
        self.conv = t.nn.Sequential(
            t.nn.Conv1d(channels, 16, 3, padding=1),
            t.nn.BatchNorm1d(16),
            t.nn.ReLU(),

            # t.nn.Conv1d(16, 32, 3, padding=1),
            # t.nn.BatchNorm1d(32),
            # t.nn.ReLU(),

            t.nn.Flatten(),
            t.nn.Linear(128, 1),
        )
        self.conv[0].bias.data.fill_(0.1)
        self.conv[0].weight.data.fill_(0.1)
        self.conv[-1].bias.data.fill_(0.1)
        self.conv[-1].weight.data.fill_(0.1)

    def forward(self, x):
        return self.conv(x)

In [5]:
from train_tools import TrainModel_NoLoader, pearson_c

model_conv = ConvModel(lookback).to(device=DEVICE)

lr = 5e-5
optimizer = t.optim.Adam(model_conv.parameters(),
                         lr=lr)
writer = SummaryWriter('./runs/1dConv/')
epochs=1800

_ = TrainModel_NoLoader(model=model_conv,
                        loss_fn=pearson_c,
                        optimizer=optimizer,
                        train_ds=train_ds,
                        val_ds=val_ds,
                        epochs=epochs,
                        display_on_epoch = 1,
                        device=DEVICE,
                        writer=writer)

2023-01-17 19:23:58.724692: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Train Loss: 0.86; Val Loss: 0.92: 100%|██████████| 1800/1800 [00:05<00:00, 337.09it/s]


In [6]:
from SeparatePredictor import SeparatePredictor

class Conv1dPredictor(SeparatePredictor):
    def prepare(self):
        self.devided_data, self.y_devided, self.class_labels = self.separate_data(self.raw_data, self.y)
        self.models = dict(zip(self.class_labels, self.models))
        self.set_up_pipeline(
            pipeline_f=Pipeline_lstm,
            pipe_params={'lookback':self.lookback},
            train_f=TrainModel_NoLoader
        )
        self.train_test_split()
        self.fit_preprocess()

        self.train_ds = self.prepare_data(self.X_train, self.y_train)
        self.test_ds = self.prepare_data(self.X_test, self.y_test)

    def train(self, device='cpu', epoch_shedule=None, epochs=100, **kwargs):
        for cls in self.class_labels:
            print(f"TRAINING f{cls}")
            writer = SummaryWriter('./runs/1dConv_multi/{}'.format(cls))
            if epoch_shedule is not None:
                epochs = epoch_shedule[cls]
            self.train_class(self.models[cls].to(device=device),
                             self.train_ds[cls],
                             self.test_ds[cls],
                             device=device,
                             writer=writer,
                             epochs=epochs,
                             **kwargs)



In [7]:
lookback = 30
epochs=2000
features = ['feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_8', 'feat_9', 'feat_10']

predictor = Conv1dPredictor(models=[ConvModel],
                            data=df,
                            params={'channels': lookback},
                            lookback=lookback,
                            device=DEVICE,
                            relevant_features=features)
predictor.prepare()

In [8]:
import json

with open('./epoch_shedule.json', 'r') as f:
    epoch_shedule = json.load(f)

In [9]:
train_params={
    'display_on_epoch':1
}
predictor.train(device=DEVICE,
                epochs=epochs,
                loss_fn=pearson_c,
                learning_rate=lr,
                epoch_shedule=epoch_shedule,
                other_train_params=train_params)

TRAINING fcoin_1


Train Loss: 0.86; Val Loss: 0.92: 100%|██████████| 1800/1800 [00:03<00:00, 498.33it/s]


TRAINING fcoin_2


Train Loss: 0.99; Val Loss: 1.00: 100%|██████████| 500/500 [00:01<00:00, 446.15it/s]


TRAINING fcoin_3


...

Train Loss: 0.80; Val Loss: 0.97: 100%|██████████| 2000/2000 [00:03<00:00, 514.15it/s]


TRAINING fcoin_64


Train Loss: 0.82; Val Loss: 0.91: 100%|██████████| 2500/2500 [00:04<00:00, 506.14it/s]


TRAINING fcoin_65


Train Loss: 0.99; Val Loss: 0.93: 100%|██████████| 800/800 [00:01<00:00, 515.20it/s]


In [10]:
ind = 28800
y_preds = predictor.predict(df.iloc[ind:], device='cuda')
t.corrcoef(t.concat([
    t.tensor(df.iloc[ind:]['fwd_ret_3d'].values).reshape(-1,1).T,
    t.tensor(y_preds.values).reshape(-1,1).T
], dim=0)
)


tensor([[1.0000, 0.0395],
        [0.0395, 1.0000]], dtype=torch.float64)

In [18]:
from datetime import datetime
y_pred = predictor.predict(df_test, DEVICE)
y_pred['Id'] = y_pred.index
y_pred['Predicted'] = y_pred[0]
y_pred.to_csv('./submission_{}.csv'.format(datetime.now().timestamp()),
              columns=['Id', 'Predicted'], index=False)

In [15]:
import datetime
datetime.datetime.now().timestamp()

1673969402.174431